In [ ]:
#Using pyhelayers package developed by IBM
!pip install pyhelayers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importing all the required packages
from keras.models import Sequential
from keras.backend import square
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Flatten, Dense
import json
import h5py
import numpy as np
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
import pyhelayers

In [ ]:
#Build a plain neural network for classifying the MNIST

# Load the MNIST dataset
(X_train_images, y_train_labels), (X_test, y_test) = mnist.load_data()

# Normalize the data
X_train_images = X_train_images / 255.0
X_test = X_test / 255.0

# Add a channel dimension of size 1
X_train_images = X_train_images.reshape((-1, 28, 28, 1)).astype('float32')
X_test = X_test.reshape((-1, 28, 28, 1)).astype('float32')


In [ ]:
# Pad the images with zeros to a size of 29x29

X_train_images = np.pad(X_train_images, ((0, 0), (0, 1), (0, 1), (0, 0)), mode='constant')
X_test = np.pad(X_test, ((0, 0), (0, 1), (0, 1), (0, 0)), mode='constant')
y_train_labels = to_categorical(y_train_labels)
y_test = to_categorical(y_test)

In [ ]:
#Split the dataset to train, test, and validation sets, which validation set is a subset of random 20 samples of the test set.
x_test, x_val, y_test, y_val = train_test_split(X_test, y_test, test_size=20, random_state=42)

# Save each dataset as an h5 file
with h5py.File("X_train_images.h5", 'w') as f:
    f.create_dataset('X_train_images', data=X_train_images)
with h5py.File("y_train_labels.h5", 'w') as f:
    f.create_dataset('y_train_labels', data=X_train_images)

with h5py.File("x_test.h5", 'w') as f:
    f.create_dataset('x_test', data=x_test)
with h5py.File("y_test.h5", 'w') as f:
    f.create_dataset('y_test', data=y_test)

with h5py.File("x_val.h5", 'w') as f:
    f.create_dataset('x_val', data=x_val)
with h5py.File("y_val.h5", 'w') as f:
    f.create_dataset('y_val', data=y_val)


In [ ]:
#Define model
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5, stride=2)
        self.fc1 = nn.Linear(in_features=5*13*13, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = x*x
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 5*13*13)
        x = self.fc1(x)
        x = x*x
        x = self.fc2(x)

        return x


In [ ]:
# Define the model
model = Sequential()
model.add(Conv2D(filters=5, kernel_size=(5, 5), strides=(2, 2), padding='same', activation=square, input_shape=(29, 29, 1)))
model.add(Flatten())
model.add(Dense(units=100, activation=square))
model.add(Dense(units=10, activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 15, 5)         130       
                                                                 
 flatten (Flatten)           (None, 1125)              0         
                                                                 
 dense (Dense)               (None, 100)               112600    
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 113,740
Trainable params: 113,740
Non-trainable params: 0
_________________________________________________________________


In [ ]:

# Train the model for 10 epochs and save the weights to a file
checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True, save_weights_only=True)
history = model.fit(X_train_images, y_train_labels, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - loss: 0.1754 - accuracy: 0.9524 - val_loss: 0.0886 - val_accuracy: 0.9739
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0648 - accuracy: 0.9809 - val_loss: 0.0625 - val_accuracy: 0.9808
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0447 - accuracy: 0.9864 - val_loss: 0.0620 - val_accuracy: 0.9815
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0348 - accuracy: 0.9890 - val_loss: 0.0649 - val_accuracy: 0.9818
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0293 - accuracy: 0.9912 - val_loss: 0.0795 - val_accuracy: 0.9824
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0246 - accuracy: 0.9922 - val_loss: 0.0791 - val_accuracy: 0.9822
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0203 - accuracy: 0.9935 - val_loss: 0.0897 - val_accuracy:

In [ ]:
# Save the model architecture to a JSON file
model_json = model.to_json()
with open('architecture.json', 'w') as f:
    f.write(model_json)

In [ ]:

# Print the final validation accuracy
val_acc = history.history['val_accuracy'][-1]
#print(f'Final validation accuracy: {val_acc:.2f}')

Final validation accuracy: 0.98


In [ ]:
nnp = pyhelayers.NeuralNetPlain()
nnp.init_arch_from_json_file("architecture.json")
nnp.init_weights_from_hdf5_file("model_weights.h5")

context = pyhelayers.DefaultContext()
optimizer = pyhelayers.HeProfileOptimizer(nnp, context)
optimizer.get_requirements().set_batch_size(16)
profile = optimizer.get_optimized_profile(False)
batch_size = profile.get_batch_size()

In [ ]:
pf1=pyhelayers.PublicFunctions()
pf1.rotate=pyhelayers.RotationSetType.CUSTOM_ROTATIONS
pf1.set_rotation_steps([10,100])

requirements = profile.requirement
requirements.public_functions=pf1


In [ ]:
context.init(profile.requirement)
print('Context ready. Batch size =',batch_size)

In [ ]:
#Encrypt the loaded model using created context.
nn = pyhelayers.NeuralNet(context)
nn.encode_encrypt(nnp, profile)

Object (detailed printing not implemented yet)

In [ ]:
#Extract batches of test samples and encrypt them
plain_samples = x_test.take(indices=range(0, batch_size), axis=0)
labels = y_test.take(indices=range(0, batch_size), axis=0)

#Perform the inference of the encrypted model, using encrypted test data.
samples = nn.encode_encrypt_input(plain_samples)

predictions=nn.predict(samples)
plain_predictions = nn.decrypt_decode_output(predictions)

In [ ]:
np.argmax(labels, axis=1)
np.argmax(plain_predictions, axis=1)


array([2, 3, 5, 2, 5, 6, 5, 1, 0, 8, 8, 5, 8, 2, 1, 2])

In [ ]:
#Latency of inference: by calculating the time difference before, and after performing the inference.

import time
import numpy as np
import tensorflow as tf

# Load the machine learning model
model = tf.keras.models.load_model('model_weights.h5')

# Define a sample input
input_data = np.array([[1, 2, 3, 4]])

# Measure the latency of inference
start_time = time.time()  # Record the start time
output = model.predict(input_data)  # Perform the inference
end_time = time.time()  # Record the end time

latency = end_time - start_time  # Calculate the latency of inference

print("Latency of inference:", latency, "seconds")
print("Model prediction:", output)


Latency of inference for batch size 16 is less compared to batch size 64 because for processing and training data of batch size 64 take more time.


In [ ]:
# Calculate the L-2 distance between the two sets of predictions
l2_distance = np.sqrt(np.sum((HE - plain) ** 2))

For the batch size of 16:
[2, 3, 5, 2, 5, 6, 5, 1, 0, 8, 8, 5, 8, 2, 1, 2]

For the batch size of 64:
[3, 7, 1, 3, 5, 1, 3, 3, 5, 2, 3, 0, 0, 3, 0, 3]

Calculating the L2 distance:

sqrt((2-3)^2 + (3-7)^2 + (5-1)^2 + (2-3)^2 + (5-5)^2 + (6-1)^2 + (5-3)^2 + (1-3)^2 + (0-5)^2 + (8-2)^2 + (8-3)^2 + (5-0)^2 + (8-0)^2 + (2-3)^2 + (1-0)^2 + (2-3)^2)

= sqrt(662)

= 25.74567

Therefore, the L2 distance between the two sets of predictions is approximately 25.74567.

Latency of inference: by calculating the time difference before, and after performing the inference.
• L-2 distance between HE and plain predictions.